In [1]:
import os
from datasets import load_dataset
import helper_utils as utils
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
import json
from datasets import load_dataset
from ultralytics import YOLO

print(tf.__version__)

/home/omar/TUM/05_projects/2D-Object-Detection/objDet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-17 16:03:52.806388: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 16:03:52.846868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 16:03:52.846914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been register

2.15.0


# 0 Data preperation

In [2]:
# Create directories for saving images and annotations
image_dir = "./football_dataset/test/images" #change the directory to train,test and val
annotations_dir = "./football_dataset/test/annotations"
if not os.path.exists(image_dir):  #check if the directory already exists
    ds = load_dataset("keremberke/football-object-detection")

    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(annotations_dir, exist_ok=True)

    # Save the dataset locally
    for i, item in enumerate(ds['test']):
        # Save image (directly save the PIL image)
        image = item['image']  # 'image' is already a PIL.JpegImageFile object
        image_path = os.path.join(image_dir, f"{i}.png")
        image.save(image_path)

        # Save annotations (e.g., bounding boxes)
        annotations = {
            "bboxes": item['objects']['bbox'],
        "object_id": item['objects']["id"]
    }
    annotations_path = os.path.join(annotations_dir, f"{i}.json")
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f)

    print(f"Saved {len(ds['train'])} images and annotations locally.")


In [3]:
# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    device = "/GPU:0"
    print("GPU is available and will be used.")
else:
    device = "/CPU:0"
    print("GPU is not available. Using CPU.")

GPU is available and will be used.


2024-10-17 16:03:57.082901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 16:03:57.084880: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-17 16:03:57.085065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
image_dir = "./football_dataset/test/images"
annotations_dir = "./football_dataset/val/annotations"
utils.convert_to_yolo_format(annotations_dir=annotations_dir,output_dir = "./yolo_football_dataset/val/labels",image_shape=(1280,720))


In [5]:

# Load a YOLOv8 model pre-trained on COCO
model = YOLO('yolov8m.pt')  # 'n' stands for nano, 's', 'm', 'l'.

# Train the model on your dataset
model.train(
    data="data.yaml",  # Define the dataset configuration in a YAML file
    epochs=100,
    imgsz=720,  # Adjust this to match your image size
    batch=4,  # Adjust based on your GPU capacity
    name='football_yolov8'  # Name the training run
)

New https://pypi.org/project/ultralytics/8.3.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=football_yolov83, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

train: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/train/labels.cache... 859 images, 1 backgrounds, 0 corrupt: 100%|██████████| 859/859 [00:00<?, ?it/s]
val: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels.cache... 243 images, 0 backgrounds, 0 corrupt: 100%|██████████| 243/243 [00:00<?, ?it/s]


Plotting labels to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 736 train, 736 val
Using 8 dataloader workers
Logging results to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.89G      1.171     0.7399      0.898        105        736: 100%|██████████| 215/215 [01:40<00:00,  2.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:07<00:00,  4.11it/s]

                   all        243       4513      0.943      0.864      0.919      0.629


...

                   all        243       4513      0.979       0.94      0.973      0.756



100 epochs completed in 3.155 hours.
Optimizer stripped from /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/last.pt, 52.0MB
Optimizer stripped from /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/best.pt, 52.0MB

Validating /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83/weights/best.pt...
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:07<00:00,  4.13it/s]


                   all        243       4513       0.98       0.94      0.974      0.756
       football_player        243       4513       0.98       0.94      0.974      0.756
Speed: 0.4ms preprocess, 22.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov83


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a14436e4d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
metrics = model.val()
print(metrics)

Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2050, 3905MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/omar/TUM/05_projects/2D-Object-Detection/yolo_football_dataset/val/labels.cache... 243 images, 0 backgrounds, 0 corrupt: 100%|██████████| 243/243 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:11<00:00,  5.40it/s]


                   all        243       4513      0.976      0.943      0.975      0.763
       football_player        243       4513      0.976      0.943      0.975      0.763
Speed: 0.5ms preprocess, 38.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /home/omar/TUM/05_projects/2D-Object-Detection/runs/detect/football_yolov832
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a1443966c80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.02202

In [8]:
results = model('football_dataset/test/images/7.png')

# Access the first result in the list
result = results[0]

# Display the results
result.show()
result.plot()  # This will plot the results on the image


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
